In [24]:
from config import Environments
from config import Buckets
from tests.setup import drop_all_tables
from pyspark.sql import SparkSession

In [25]:
environment = Environments.LOCAL
bucket = Buckets[environment.value].value
table_name = "my_table"
table_path = f"{bucket}/{table_name}"

print(f"{environment=}")
print(f"{bucket=}")
print(f"{table_path=}")

environment=<Environments.LOCAL: 'LOCAL'>
bucket='/tmp/deltalake'
table_path='/tmp/deltalake/my_table'


In [26]:
drop_all_tables()

2024-03-11 20:33:52.684 | INFO     | tests.setup:drop_all_tables:49 - dropping table(s) in /tmp/deltalake
2024-03-11 20:33:52.685 | WARNING  | tests.setup:drop_all_tables:55 - [Errno 2] No such file or directory: '/tmp/deltalake'


In [27]:


spark = (
    SparkSession
    .builder
    .appName("DeltaLogApp")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog",)
    .getOrCreate()
)

In [37]:
# Sample data #1
# data = [("Alice", 25), ("Bob", 30), ("Jake", 59)]
# data = [("Kate", 44), ("Bob", 30), ("Jake", 100)]
data = [("Kate", 44, "Smooth Alice"), ("Bob", 30, "Steezy Bob"), ("Jake", 100, "Speedy Jake")]

# Define a schema
# schema = """name:string, age:integer"""
schema = """name:string, age:integer, nickname:string"""

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)

df.show()

+----+---+------------+
|name|age|    nickname|
+----+---+------------+
|Kate| 44|Smooth Alice|
| Bob| 30|  Steezy Bob|
|Jake|100| Speedy Jake|
+----+---+------------+



In [38]:
# Write data to a Delta table
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
df.repartition(1).write.format("delta").mode("overwrite").save(table_path)

In [ ]:
# Read the Delta table
delta_df = spark.read.format("delta").load(table_path)

In [39]:
# Display the Delta Log
delta_log_df = spark.read.format("delta").option("versionAsOf", 2).load(f"{table_path}")
delta_log_df.show(truncate=False)

+----+---+------------+
|name|age|nickname    |
+----+---+------------+
|Kate|44 |Smooth Alice|
|Bob |30 |Steezy Bob  |
|Jake|100|Speedy Jake |
+----+---+------------+



In [ ]:
# Display the Delta Log
delta_log_df = spark.read.format("json").load(f"{table_path}/_delta_log")
delta_log_df.show(truncate=True)